In [2]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Define parameters
MaxBatchSize = 10
MaxBatchTime = 15  # in minutes
OrderTimeWindow = 10  # in minutes
alpha = 0.75  # weight for emissions
beta = 0.25  # weight for batching

# Sample data (you should replace this with your actual data)
customers = [1, 2, 3, 4, 5]
demands = {1: 1, 2: 2, 3: 1, 4: 3, 5: 2}
travel_time = {(i, j): 10 for i in customers for j in customers if i != j}
emission_factor = {(i, j): 0.1 for i in customers for j in customers if i != j}

# Set an arbitrary value for the emission constraint (you should replace this with your actual data)
Emax = 10

# Create LP problem for determining the optimum batch size
batch_size_prob = LpProblem("Optimal_Batch_Size", LpMinimize)

# Decision variables
x = LpVariable.dicts("x", [(i, j) for i in customers for j in customers if i != j], cat="Binary")
y = LpVariable.dicts("y", customers, cat="Binary")
BatchSize = LpVariable("BatchSize", lowBound=1, upBound=MaxBatchSize, cat="Continuous")
u = LpVariable.dicts("u", customers, lowBound=0, upBound=MaxBatchSize, cat="Continuous")

# Objective function
batch_size_prob += (
    lpSum(travel_time[i, j] * x[i, j] for i in customers for j in customers if i != j) +
    alpha * lpSum(emission_factor[i, j] * demands[i] * x[i, j] for i in customers for j in customers if i != j) +
    beta * BatchSize
)

# Max weight per batch
MaxWeightPerBatch = 50

# Constraints
batch_size_prob += lpSum(y[i] * demands[i] for i in customers) <= MaxWeightPerBatch  # Batch weight constraint
batch_size_prob += lpSum(travel_time[i, j] * x[i, j] for i in customers for j in customers if i != j) <= MaxBatchTime  # Batch time constraint
batch_size_prob += lpSum(emission_factor[i, j] * demands[i] * x[i, j] for i in customers for j in customers if i != j) <= Emax  # Emission constraint

# Position assignment for customers in a batch
for i in customers:
    for j in customers:
        if i != j:
            batch_size_prob += u[i] - u[j] + MaxBatchSize * x[i, j] <= MaxBatchSize - 1  # Position assignment for customers in a batch

            # Ensure that a batch is selected only if any customer in that batch is selected
            batch_size_prob += BatchSize >= y[i]
# Solve the problem
batch_size_prob.solve()

# Print results
print("Status:", batch_size_prob.status)
print("Optimal Batch Size:", round(BatchSize.varValue, 2))



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/amarigarrett/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/lt/njk9c7vd43n4xfmcy1kt75c40000gn/T/0f8a9956d3454fadab61c49b69d37574-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/lt/njk9c7vd43n4xfmcy1kt75c40000gn/T/0f8a9956d3454fadab61c49b69d37574-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 48 COLUMNS
At line 265 RHS
At line 309 BOUNDS
At line 342 ENDATA
Problem MODEL has 43 rows, 31 columns and 145 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.25 - 0.00 seconds
Cgl0004I processed model has 20 rows, 5 columns (0 integer (0 of which binary)) and 40 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0.25 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of whi

In [3]:
import random
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Simulated data
random.seed(5)  # For reproducibility

# Constants
OrderTimeWindow = 10  # in minutes
alpha = 1  # weight for emissions
beta = 1  # weight for batching

# Simulated traffic speeds (km/h)
best_case_speed_range = (40, 50)
average_speed = 45
low_speed= (1,40)
high=(51,80)

# Assuming you have a function to calculate travel time based on distance and speed
def calculate_travel_time(customer1, customer2, speed):
    # Your logic to calculate travel time based on distance and speed
    distance = ((customer1["x"] - customer2["x"])**2 + (customer1["y"] - customer2["y"])**2)**0.5
    return distance / speed

# Generate random orders with coordinates
def generate_orders(num_orders):
    orders = []
    for _ in range(num_orders):
        order = {
            "customer_id": random.randint(1, 100),
            "order_weight": random.uniform(1, 5),  # Weight in kg
            "x": random.uniform(0, 100),  # X-coordinate
            "y": random.uniform(0, 100),  # Y-coordinate
        }
        orders.append(order)
    return orders

# Traffic scenarios
def simulate_traffic_scenario(speed_range):
    best_case_speed = random.uniform(speed_range[0], speed_range[1])
    print(f"\nSimulating best-case traffic scenario with speed: {best_case_speed} km/h...")

    total_delivery_cost = 0
    total_emissions = 0

    for iteration in range(1, 31):
        print(f"\nIteration {iteration}")

        # Generate random orders with coordinates for each iteration
        orders = generate_orders(200)

        # Create LP problem for determining the optimal batch size
        batch_size_prob = LpProblem(f"Optimal_Batch_Size_Best_Case_Iteration_{iteration}", LpMinimize)

        # Dynamic constraints
        MaxVehicleCapacity = 40  # Example: Maximum capacity of a delivery vehicle in terms of order weight
        MaxOperationTime = 180  # Example: Maximum time for a delivery operation in minutes

        # Decision variables
        x = LpVariable.dicts("x", [(i["customer_id"], j["customer_id"]) for i in orders for j in orders if i != j], cat="Binary")
        y = LpVariable.dicts("y", [i["customer_id"] for i in orders], cat="Binary")
        BatchSize = LpVariable("BatchSize", lowBound=1, upBound=MaxVehicleCapacity, cat="Continuous")
        u = LpVariable.dicts("u", [i["customer_id"] for i in orders], lowBound=0, upBound=MaxVehicleCapacity, cat="Continuous")

        # Objective function
        batch_size_prob += (
            lpSum((i["order_weight"] + j["order_weight"]) * x[i["customer_id"], j["customer_id"]] for i in orders for j in orders if i != j) +
            alpha * lpSum(i["order_weight"] * x[i["customer_id"], j["customer_id"]] for i in orders for j in orders if i != j) +
            beta * BatchSize
        )

        # Constraints
        batch_size_prob += lpSum(y[i["customer_id"]] for i in orders) <= BatchSize  # Batch size constraint
        batch_size_prob += lpSum(calculate_travel_time(i, j, best_case_speed) * x[i["customer_id"], j["customer_id"]] for i in orders for j in orders if i != j) <= MaxOperationTime  # Batch time constraint

        # Solve the problem
        batch_size_prob.solve()

        # Calculate metrics
        total_delivery_cost += batch_size_prob.objective.value()
        total_emissions += alpha * batch_size_prob.objective.value()

        # Print results for each iteration
        print("Status:", batch_size_prob.status)
        print("Optimal Batch Size:", round(BatchSize.varValue, 2))
        print("Total Delivery Cost:", round(batch_size_prob.objective.value(), 2))
        print("Total Emissions:", round(alpha * batch_size_prob.objective.value(), 2))

    # Print average results for the best-case scenario
    avg_delivery_cost = total_delivery_cost / 30
    avg_emissions = total_emissions / 30
    print(f"\nAverage Delivery Cost (Best Case): {round(avg_delivery_cost, 2)}")
    print(f"Average Emissions (Best Case): {round(avg_emissions, 2)}")

# Run simulation for the best-case traffic scenario
simulate_traffic_scenario(best_case_speed_range)

# Run simulation for the best-case traffic scenario
simulate_traffic_scenario(low_speed)

# Run simulation for the best-case traffic scenario
simulate_traffic_scenario(high)


Simulating best-case traffic scenario with speed: 46.22901694889702 km/h...

Iteration 1
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/amarigarrett/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/lt/njk9c7vd43n4xfmcy1kt75c40000gn/T/a9ab15b636c74725b8c4df873df4eed8-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/lt/njk9c7vd43n4xfmcy1kt75c40000gn/T/a9ab15b636c74725b8c4df873df4eed8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 30431 RHS
At line 30434 BOUNDS
At line 38064 ENDATA
Problem MODEL has 2 rows, 7628 columns and 7628 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node chan